In [1]:
import pandas as pd
import re

import past_guest_module as pm

import sys
sys.path.append('../zoho_data_app/')

import main_module as m

%load_ext autoreload
%autoreload 2


In [2]:
path = r'C:\Users\fajar\Documents\Python\Data\check_country_code_analytic.xlsx'
df_analytic = pd.read_excel(path, dtype='string', sheet_name='Sheet1')
count_data_awal = df_analytic.shape[0]
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK063726,Karma Resorts,Other,Past Guests,Karma Kandara,PF292364,Mr,Ahmed,Almalki,Male,...,2024-11-11 00:00:00,1,HOUSE_USE,V_OWNER_NRB,COMP,3,0,0,Email and Phone,NO
1,KK056195,Karma Resorts,Other,Past Guests,Karma Kandara,PF274675,Mr,Mark,Bayliss,Male,...,2024-11-09 00:00:00,8,DIGITAL,WEBSITE,DIGITAL,2,0,0,Email and Phone,NO
2,KK056260,Karma Resorts,Other,Past Guests,Karma Kandara,PF274971,Ms,Paula Tini Pouaru,Luke,Unknown,...,2024-11-12 00:00:00,7,DIGITAL,WEBSITE,DIGITAL,5,0,0,Email and Phone,NO
3,KK057597,Karma Resorts,Other,Past Guests,Karma Kandara,PF278508,Mr,Kristian,Vestergaard,Male,...,2024-11-10 00:00:00,6,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
4,KK048775,Karma Resorts,Other,Past Guests,Karma Kandara,PF260355,Ms,Izzy,Burgess,Female,...,2024-11-09 00:00:00,7,HOUSE_USE,V_OWNER,COMP,4,0,0,Email Only,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,KK063791,Karma Resorts,Other,Past Guests,Karma Kandara,PF292456,Mr,Daniel,Jimenez,Unknown,...,2024-11-10 00:00:00,2,HOUSE_USE,V_OWNER,COMP,2,0,0,Email Only,NO
79,KB045185,Karma Resorts,Other,Past Guests,Karma Bavaria,PF290781,Herr,Genc,Turku,Unknown,...,2024-11-10 00:00:00,2,LGR,LGR,GROUP,2,0,0,Email Only,NO
80,KB042915,Karma Resorts,Other,Past Guests,Karma Bavaria,<NA>,Frau,Angelika,Richter,Unknown,...,2024-11-14 18:00:00,0,CB,CB,GROUP,1,0,0,Email Only,NO
81,KK063781,Karma Resorts,Other,Past Guests,Karma Kandara,PF292453,Ms,Cynthia,Tio,Unknown,...,2024-11-09 00:00:00,2,HOUSE_USE,V_OWNER_NRB,COMP,2,0,0,Email Only,NO


In [3]:
df_analytic.columns

Index(['Booking_Id', 'Lead Brand', 'Lead Sub- Brand', 'Lead Source',
       'Resort_Name', 'ProfileRef', 'Salutation', 'First_Name', 'Surname',
       'Gender', 'Language', 'DOB', 'Street', 'Area', 'Town', 'County',
       'Country', 'Nationality', 'PostCode', 'TelNo', 'MobileNo', 'Email',
       'Exclude From Mailing', 'Opt In', 'Booking Status', 'DateArrival',
       'DateDepart', 'No. of Nights', 'Market_Segment', 'Media_Code',
       'Media Code Group', 'No. of Adult', 'No. of Children', 'No. of Infants',
       'Contact Type', 'Odyssey Members'],
      dtype='object')

In [4]:
df_analytic.fillna('empty', inplace=True)

In [5]:
path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Phone Members.xlsx'
df_phone = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Email Members.xlsx'
df_email = pd.read_excel(path, dtype='string', sheet_name='Sheet1')


In [6]:
m.clean_space(df_email, 'Email')
df_email['Email'] = m.lowercase(df_email, 'Email')

c:\Users\fajar\Documents\Python\Task Offline\past_guest_toni_app\../zoho_data_app\main_module.py:90: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[df_columns].replace(' ','', regex=True, inplace=True)


In [7]:
# hitung jumlah duplicate
count_duplicate_general = df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])].shape[0]

df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])]

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
7,KB045283,Karma Resorts,Other,Past Guests,Karma Bavaria,PF284976,Frau,Dhurata,Schauer,Female,...,2024-11-09 23:30:00,0,LGR,LGR,GROUP,1,0,0,Email and Phone,NO
8,KB044264,Karma Resorts,Other,Past Guests,Karma Bavaria,PF284976,Frau,Dhurata,Schauer,Female,...,2024-11-09 21:00:00,0,LGR,LGR,GROUP,1,0,0,Email and Phone,NO
9,KB044996,Karma Resorts,Other,Past Guests,Karma Bavaria,PF284976,Frau,Dhurata,Schauer,Female,...,2024-11-10 10:30:00,0,LGR,LGR,GROUP,1,0,0,Email and Phone,NO
13,KB043171,Karma Resorts,Other,Past Guests,Karma Bavaria,PF278714,Frau,Lobna,Magdy,Female,...,2024-11-08 18:00:00,0,CB,CB,GROUP,1,0,0,Email and Phone,NO
15,KB044405,Karma Resorts,Other,Past Guests,Karma Bavaria,PF284939,Frau,Nadine,Remmert,Female,...,2024-11-09 23:00:00,0,LGR,LGR,GROUP,1,0,0,Email and Phone,NO
16,KB044404,Karma Resorts,Other,Past Guests,Karma Bavaria,PF284939,Frau,Nadine,Remmert,Female,...,2024-11-09 23:00:00,0,LGR,LGR,GROUP,1,0,0,Email and Phone,NO
21,KB044524,Karma Resorts,Other,Past Guests,Karma Bavaria,PF286668,Herr,Sebastien,Raguin,Unknown,...,2024-11-10 00:00:00,2,COMP,GER_OWE,COMP,2,2,0,Email Only,NO
22,KB045374,Karma Resorts,Other,Past Guests,Karma Bavaria,PF080008,empty,Kea,Marketing Gmbh,Unknown,...,2024-11-10 19:00:00,0,LGR,LGR,GROUP,1,0,0,Email Only,NO
23,KB044595,Karma Resorts,Other,Past Guests,Karma Bavaria,PF287238,Mrs,Anjie,Guo,Unknown,...,2024-11-10 00:00:00,2,COMP,GER_OWE,COMP,4,0,0,Email Only,NO
24,KB040422,Karma Resorts,Other,Past Guests,Karma Bavaria,PF080008,empty,Kea,Marketing Gmbh,Unknown,...,2024-11-10 19:30:00,0,LGR,LGR,GROUP,1,0,0,Email Only,NO


In [8]:
df_analytic.drop_duplicates(subset=['TelNo','MobileNo','Email'], inplace=True)
df_analytic.reset_index(drop=True, inplace=True)

# hitung jumlah setelah value duplicate dihapus
count_after_dedup_duplicate = df_analytic.shape[0]

df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK063726,Karma Resorts,Other,Past Guests,Karma Kandara,PF292364,Mr,Ahmed,Almalki,Male,...,2024-11-11 00:00:00,1,HOUSE_USE,V_OWNER_NRB,COMP,3,0,0,Email and Phone,NO
1,KK056195,Karma Resorts,Other,Past Guests,Karma Kandara,PF274675,Mr,Mark,Bayliss,Male,...,2024-11-09 00:00:00,8,DIGITAL,WEBSITE,DIGITAL,2,0,0,Email and Phone,NO
2,KK056260,Karma Resorts,Other,Past Guests,Karma Kandara,PF274971,Ms,Paula Tini Pouaru,Luke,Unknown,...,2024-11-12 00:00:00,7,DIGITAL,WEBSITE,DIGITAL,5,0,0,Email and Phone,NO
3,KK057597,Karma Resorts,Other,Past Guests,Karma Kandara,PF278508,Mr,Kristian,Vestergaard,Male,...,2024-11-10 00:00:00,6,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
4,KK048775,Karma Resorts,Other,Past Guests,Karma Kandara,PF260355,Ms,Izzy,Burgess,Female,...,2024-11-09 00:00:00,7,HOUSE_USE,V_OWNER,COMP,4,0,0,Email Only,NO
5,KB043226,Karma Resorts,Other,Past Guests,Karma Bavaria,PF278932,Frau,Angelika,Hauser,Female,...,2024-11-11 00:00:00,2,DIR,TEL,RETAIL,7,0,0,Email and Phone,NO
6,KB044263,Karma Resorts,Other,Past Guests,Karma Bavaria,PF284976,Frau,Dhurata,Schauer,Female,...,2024-11-09 23:55:00,0,LGR,LGR,GROUP,1,0,0,Email and Phone,NO
7,KB045574,Karma Resorts,Other,Past Guests,Karma Bavaria,PF192683,empty,Kathrin,Haas,Female,...,2024-11-08 00:00:00,2,COMP,GER_OWE,COMP,2,0,0,Email Only,NO
8,KB038479,Karma Resorts,Other,Past Guests,Karma Bavaria,PF254425,Frau,Jeannette,Matthews,Female,...,2024-11-11 20:30:00,0,DIR,WALKIN,RETAIL,1,0,0,Email Only,NO
9,KB043170,Karma Resorts,Other,Past Guests,Karma Bavaria,PF278714,Frau,Lobna,Magdy,Female,...,2024-11-07 18:00:00,0,CB,CB,GROUP,1,0,0,Email and Phone,NO


In [9]:
find_member_booking = df_analytic.copy()

find_member_booking.loc[find_member_booking['TelNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile' 
find_member_booking.loc[find_member_booking['MobileNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile'

find_member_booking.loc[find_member_booking['Email'].isin(df_email['Email']), 'Same Value Find'] = 'Member-Email'
find_member_booking.drop(find_member_booking.loc[find_member_booking['Same Value Find'].isna()].index, inplace=True)

find_member_booking.replace('empty', '', inplace=True)

In [10]:
find_member_booking.to_csv(r'C:\Users\fajar\Documents\Python\Data\cek_member_zoho_analytic.csv', index=False)

In [11]:
count_dedup_member = df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])].shape[0]


df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])]


,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
2,KK056260,Karma Resorts,Other,Past Guests,Karma Kandara,PF274971,Ms,Paula Tini Pouaru,Luke,Unknown,...,2024-11-12 00:00:00,7,DIGITAL,WEBSITE,DIGITAL,5,0,0,Email and Phone,NO
3,KK057597,Karma Resorts,Other,Past Guests,Karma Kandara,PF278508,Mr,Kristian,Vestergaard,Male,...,2024-11-10 00:00:00,6,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
7,KB045574,Karma Resorts,Other,Past Guests,Karma Bavaria,PF192683,empty,Kathrin,Haas,Female,...,2024-11-08 00:00:00,2,COMP,GER_OWE,COMP,2,0,0,Email Only,NO
12,KB044525,Karma Resorts,Other,Past Guests,Karma Bavaria,PF216954,Mr,David,Raguin,Male,...,2024-11-10 00:00:00,2,COMP,GER_OWE,COMP,2,0,2,Email and Phone,NO


In [12]:
df_analytic.drop(df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])
].index, inplace=True)

df_analytic.reset_index(drop=True, inplace=True)

count_after_dedup_member = df_analytic.shape[0]

In [13]:
count_data_akhir = df_analytic.shape[0]

In [14]:
df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KSH019923,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF285647,Mr,Rishi,Chauhan,Male,...,2024-11-07 00:00:00,1,DIR,WEBSITE,DIGITAL,2,0,0,Email and Phone,NO
1,KK056920,Karma Resorts,Other,Past Guests,Karma Kandara,PF276716,Mrs,Rakesh,Chauhan,Female,...,2024-11-07 00:00:00,6,DIGITAL,WEBSITE,DIGITAL,2,0,0,Email and Phone,NO
2,KB045021,Karma Resorts,Other,Past Guests,Karma Bavaria,PF287030,empty,Eva,Rubenthaler,Unknown,...,2024-11-07 00:00:00,3,OTA,OTA,RETAIL,1,0,0,Email and Phone,NO
3,KK062767,Karma Resorts,Other,Past Guests,Karma Kandara,PF283420,Mr,Imanuel Handjaja,Ong,Male,...,2024-11-07 00:00:00,1,DIGITAL,CHAT,DIGITAL,3,0,0,Email and Phone,NO
4,KB043170,Karma Resorts,Other,Past Guests,Karma Bavaria,PF278714,Frau,Lobna,Magdy,Female,...,2024-11-07 18:00:00,0,CB,CB,GROUP,1,0,0,Email and Phone,NO
5,KK060817,Karma Resorts,Other,Past Guests,Karma Kandara,PF241488,Mr,Imanuel Handjaja,Ong,Male,...,2024-11-08 00:00:00,3,HOUSE_USE,V_OWNER,COMP,9,0,0,Email and Phone,NO
6,KK056195,Karma Resorts,Other,Past Guests,Karma Kandara,PF274675,Mr,Mark,Bayliss,Male,...,2024-11-09 00:00:00,8,DIGITAL,WEBSITE,DIGITAL,2,0,0,Email and Phone,NO
7,KK048775,Karma Resorts,Other,Past Guests,Karma Kandara,PF260355,Ms,Izzy,Burgess,Female,...,2024-11-09 00:00:00,7,HOUSE_USE,V_OWNER,COMP,4,0,0,Email Only,NO
8,KLM006546,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF284944,Mrs,Catherine,Clark,Female,...,2024-11-09 00:00:00,1,DIR,WEBSITE,DIGITAL,2,0,0,Email and Phone,NO
9,KK056493,Karma Resorts,Other,Past Guests,Karma Kandara,PF275524,Mr,Sujith,Kumar,Male,...,2024-11-09 00:00:00,3,OTA,OTA,RETAIL,3,0,0,Email and Phone,NO


In [15]:
df_analytic['DOB'].loc[df_analytic['DOB']=='']

Series([], Name: DOB, dtype: string)

In [16]:
df_analytic['DOB'].replace('empty', '', inplace=True)
dob_con_date = pd.to_datetime(df_analytic['DOB'], errors='coerce')
dob_date = dob_con_date.dt.strftime('%Y-%m-%d')

arr_con_date = pd.to_datetime(df_analytic['DateArrival'])
arr_date = arr_con_date.dt.strftime('%Y-%m-%d')

dep_con_date = pd.to_datetime(df_analytic['DateDepart'])
dep_date = dep_con_date.dt.strftime('%Y-%m-%d')

df_analytic['DOB'] = dob_date
df_analytic['DateArrival'] = arr_date
df_analytic['DateDepart'] = dep_date


C:\Users\fajar\AppData\Local\Temp\ipykernel_15388\1513714307.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_analytic['DOB'].replace('empty', '', inplace=True)


In [17]:
df_analytic.replace('empty', '', inplace=True)

df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KSH019923,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF285647,Mr,Rishi,Chauhan,Male,...,2024-11-07,1,DIR,WEBSITE,DIGITAL,2,0,0,Email and Phone,NO
1,KK056920,Karma Resorts,Other,Past Guests,Karma Kandara,PF276716,Mrs,Rakesh,Chauhan,Female,...,2024-11-07,6,DIGITAL,WEBSITE,DIGITAL,2,0,0,Email and Phone,NO
2,KB045021,Karma Resorts,Other,Past Guests,Karma Bavaria,PF287030,,Eva,Rubenthaler,Unknown,...,2024-11-07,3,OTA,OTA,RETAIL,1,0,0,Email and Phone,NO
3,KK062767,Karma Resorts,Other,Past Guests,Karma Kandara,PF283420,Mr,Imanuel Handjaja,Ong,Male,...,2024-11-07,1,DIGITAL,CHAT,DIGITAL,3,0,0,Email and Phone,NO
4,KB043170,Karma Resorts,Other,Past Guests,Karma Bavaria,PF278714,Frau,Lobna,Magdy,Female,...,2024-11-07,0,CB,CB,GROUP,1,0,0,Email and Phone,NO
5,KK060817,Karma Resorts,Other,Past Guests,Karma Kandara,PF241488,Mr,Imanuel Handjaja,Ong,Male,...,2024-11-08,3,HOUSE_USE,V_OWNER,COMP,9,0,0,Email and Phone,NO
6,KB044263,Karma Resorts,Other,Past Guests,Karma Bavaria,PF284976,Frau,Dhurata,Schauer,Female,...,2024-11-09,0,LGR,LGR,GROUP,1,0,0,Email and Phone,NO
7,KB044403,Karma Resorts,Other,Past Guests,Karma Bavaria,PF284939,Frau,Nadine,Remmert,Female,...,2024-11-09,0,LGR,LGR,GROUP,1,0,0,Email and Phone,NO
8,KK054963,Karma Resorts,Other,Past Guests,Karma Kandara,PF271907,Mr,Edwin,Edwin,Male,...,2024-11-09,4,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
9,KK061805,Karma Resorts,Other,Past Guests,Karma Kandara,PF232826,Mr,Dylan,Tan,Male,...,2024-11-09,2,HOUSE_USE,V_OWNER,COMP,2,0,0,Email and Phone,NO


In [18]:
print("==========================================================")
print("Summary")
print("==========================================================")
print("Jumlah Data Setelah Script Past Guest Sebelumnya : "+str(count_data_awal))
print("Jumlah Data Yang Duplicate : "+str(count_duplicate_general))
print("Jumlah Data Setelah Hapus Duplicate : "+str(count_after_dedup_duplicate))
print("Jumlah Data Yang Termasuk Member: "+str(count_dedup_member))
print("Jumlah Data Setelah Hapus Member : "+str(count_after_dedup_member))
print("Jumlah Data Akhir : "+str(count_data_akhir))

Summary
Jumlah Data Setelah Script Past Guest Sebelumnya : 83
Jumlah Data Yang Duplicate : 47
Jumlah Data Setelah Hapus Duplicate : 36
Jumlah Data Yang Termasuk Member: 4
Jumlah Data Setelah Hapus Member : 32
Jumlah Data Akhir : 32


In [19]:
df_analytic.to_excel(r'C:\Users\fajar\Documents\Python\Data\final_analytics.xlsx', index=False)